In [1]:
from translator import translate
from transformers import pipeline
import pandas as pd
import numpy as np
import os
from src.functions import check,tresh,license_plate
import unittest
import sys
import operator
from tqdm import tqdm
import nltk


2022-04-28 13:30:54.608211: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-28 13:30:54.608229: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from datasets import load_dataset

dataset = load_dataset('xtreme', 'XNLI', split='test')
df=pd.DataFrame(dataset)
df

2022-04-28 13:30:57,195 - WARNING - download_and_prepare - Reusing dataset xtreme (/home/nour/.cache/huggingface/datasets/xtreme/XNLI/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17) 


,language,sentence1,sentence2,gold_label
0,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,لم أتحدث معه مرة أخرى.,contradiction
1,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,كنت مستاء جدا لدرجة أنني بدأت بالحديث معه مرة ...,entailment
2,ar,حسنا ، لم أكن أفكر حتى حول ذلك ، لكن كنت محبطا...,دار بيننا حديث رائع.,neutral
3,ar,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,لم أكن على علم بأنني لم أكن الشخص الوحيد الذي ...,neutral
4,ar,واعتقدت أن ذلك شرف لي ، ولا يزال ، ولايزال ، ك...,كان لدي إنطباع أنني كنت الشخص الوحيد الذي لديه...,entailment
...,...,...,...,...
75145,zh,戴维森不应该用“scone”的发音来和“bone”押韵 - 无论如何不是因为维多利亚，而是他...,戴维森不应该以骨头和烤饼听起来一样的方式说话。,entailment
75146,zh,戴维森不应该用“scone”的发音来和“bone”押韵 - 无论如何不是因为维多利亚，而是他...,"如果戴维森押韵饼和骨头的话, 那就更好了。",contradiction
75147,zh,20万字的小说平均每8000字25美元。,价值25美元的20万字小说价格合理。,neutral
75148,zh,20万字的小说平均每8000字25美元。,"25美元的20万字的小说就是每美元4,000字。",contradiction


In [3]:
df_fr=df[df.language=='fr']
df_fr

,language,sentence1,sentence2,gold_label
30060,fr,"Eh bien, je ne pensais même pas à cela, mais j...",Je ne lui ai pas parlé de nouveau,contradiction
30061,fr,"Eh bien, je ne pensais même pas à cela, mais j...",J'étais si contrarié que je commençais juste à...,entailment
30062,fr,"Eh bien, je ne pensais même pas à cela, mais j...",Nous avons eu une grande discussion.,neutral
30063,fr,"Et je pensais que c'était un privilège, et ça ...",Je n'avais pas conscience que je n'étais pas l...,neutral
30064,fr,"Et je pensais que c'était un privilège, et ça ...",J'avais l'impression que j'étais le seul à avo...,entailment
...,...,...,...,...
35065,fr,Davidson ne devrait pas adopter la prononciati...,Davidson ne devrait pas parler d'une façon où ...,entailment
35066,fr,Davidson ne devrait pas adopter la prononciati...,Ce serait mieux que Davidson fasse rimer les m...,contradiction
35067,fr,Le roman moyen de 200 000 mots pour 25 $ fonct...,Un roman de 200 000 mots à 25 $ est un prix éq...,neutral
35068,fr,Le roman moyen de 200 000 mots pour 25 $ fonct...,Un roman de 200 000 payé 25 $ revient à 4 000 ...,contradiction


In [4]:
df_en=df[df.language=='en']
df_en

,language,sentence1,sentence2,gold_label
20040,en,"Well, I wasn't even thinking about that, but I...",I havent spoken to him again.,contradiction
20041,en,"Well, I wasn't even thinking about that, but I...",I was so upset that I just started talking to ...,entailment
20042,en,"Well, I wasn't even thinking about that, but I...",We had a great talk.,neutral
20043,en,"And I thought that was a privilege, and it's s...",I was not aware that I was not the only person...,neutral
20044,en,"And I thought that was a privilege, and it's s...",I was under the impression that I was the only...,entailment
...,...,...,...,...
25045,en,Davidson should not adopt the pronunciation of...,Davidson shouldn't talk in a way where bone an...,entailment
25046,en,Davidson should not adopt the pronunciation of...,It would be better if Davidson rhymed the word...,contradiction
25047,en,"The average novel of 200,000 words for $25 wor...","A 200,000 word novel at $25 is a fair price.",neutral
25048,en,"The average novel of 200,000 words for $25 wor...","A 200,000 word novel for $25 is 4,000 words pe...",contradiction


In [5]:
pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [6]:
# # candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
# name=["First and last name","full name", "person's name","first name and Family name","person"]
# birth=["birthday"]
# # ip=["Internet protocol (IP) addresses","IP adress"]
# cookie=["cookie identifiers","a cookie"]
# phone=["phone numbers","phone number","telphone number"]
# mail=["mail addresses","mail address","mail"]
# location=["location adress","private postal address"]
# health=["health data","medical records","health information","medical information","Medical Procedure names","Medical Procedure identification codes"]
# personal=["personal data","personal","personal information","private data","private information"]
# not_personal=["not personal data","not personal information","not personal","not private data","not private","not private information"]
# passport=['passport','code']
# Driving_license=['driving license','code']
# social_security_number=['social security number']
# Tax_file_number=['tax file number']
# credit_card=['credit card number']
# candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passport+Driving_license+social_security_number+Tax_file_number+credit_card

In [10]:
# candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
name=["Nom et prenom","name and family name","person's name"]
birth=['birthday','private']
# ip=["Internet protocol (IP) addresses","IP adress"]
cookie=["identifiants de cookie"]
phone=["numéros de téléphone","phone number"]
mail=["courrier electronique",'mail','email']
location=["location adress","private postal address",'adresse postale']
health=["données de santé", "dossiers médicaux", " médicales", "noms des actes médicaux", "codes d'identification des actes médicaux","Medical Procedure identification codes"]
personal=["données personnelles", "personnel", "informations personnelles", "données privées", "informations privées"]
not_personal=["données non personnelles", "informations non personnelles", "non personnel", "données non privées", "non privé", "informations non privées"]
passeport=['passport']
Driving_license=['numero de permis','driving license']
social_security_number=['social security number']
Tax_file_number=['numéro de dossier fiscal']
credit_card=['numéro de carte de crédit']
candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passeport+Driving_license+social_security_number+Tax_file_number+credit_card

In [12]:
text_for_test="""Marié à Michelle Robinson en 1992, Barack Obama entre en politique quatre ans plus tard : il est élu au Sénat de l'Illinois, où il effectue trois mandats, de 1997 à 2004.
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 22s {'person': 0.8287651658058166}

100%|██████████| 1/1 [00:07<00:00,  7.35s/it]

Marié à Michelle Robinson en 1992, Barack Obama entre en politique quatre ans plus tard : il est élu au Sénat de l'Illinois, où il effectue trois mandats, de 1997 à 2004.
{'person': 0.7697925170262655} 



In [13]:
text_for_test="""un homme d'État américain. Il est le 44e président des États-Unis, en fonction du 20 janvier 2009 au 20 janvier 2017.
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        text_translated=translate(txt)
        result_details=pipeline(text_translated, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 38s No PII detected / No PII detected

 50%|█████     | 1/2 [00:19<00:19, 19.60s/it]

un homme d'État américain.
No PII detected


100%|██████████| 2/2 [00:40<00:00, 20.28s/it]

Il est le 44e président des États-Unis, en fonction du 20 janvier 2009 au 20 janvier 2017.
No PII detected


In [14]:
text_for_test=""" Toto TITI a fait de la natation pendant 5 ans, entre temps il a refait son passeport de numero 215658125 au consulat
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 20s {'person': 0.9293522715568543, 'passport': 0.5010863989591599}

100%|██████████| 1/1 [00:06<00:00,  6.98s/it]

 Toto TITI a fait de la natation pendant 5 ans, entre temps il a refait son passeport de numero 215658125 au consulat
passport checked
Driving_license checked
{'passport': 0.9895483255386353, 'person': 0.677378449589014, 'location': 0.6000065604845682, 'Driving_license': 0.7497215569019318} 



In [15]:
text_for_test=""" la plaque d'immatriculation de ma voiture a été volé
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 19s No PII detected

100%|██████████| 1/1 [00:04<00:00,  4.98s/it]

 la plaque d'immatriculation de ma voiture a été volé
No PII detected


In [16]:
text_for_test=""" le numero de la plaque d'immatriculation de ma voiture est AA-123-BB
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 20s {'Driving_license': 0.6713268160820007, 'license_plate': 0.9}

100%|██████████| 1/1 [00:05<00:00,  5.16s/it]

 le numero de la plaque d'immatriculation de ma voiture est AA-123-BB
{'license_plate': 0.9} 



In [17]:
text_for_test=""" le numero de securite est 2000699352685
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 21s {'social_security_number': 0.8903656601905823}

100%|██████████| 1/1 [00:06<00:00,  6.23s/it]

 le numero de securite est 2000699352685
{'social_security_number': 0.8842569589614868} 



In [18]:
text_for_test=""" Kired Nour Elhouda est née le 28 juin 20000 à Saida Algerie
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 21s {'social_security_number': 0.8903656601905823}

100%|██████████| 1/1 [00:05<00:00,  5.79s/it]

 Kired Nour Elhouda est née le 28 juin 20000 à Saida Algerie
birth checked
{'person': 0.8161532282829285, 'birth': 0.5106632728129625} 



In [19]:
text_for_test=""" you can find my passport in the back of my car, on the left side. You can call me on  +33(0)6 12 34 56 78
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 26s No PII detected/{'phone': 0.9922093947728475}

 50%|█████     | 1/2 [00:06<00:06,  6.11s/it]

 you can find my passport in the back of my car, on the left side.
No PII detected


100%|██████████| 2/2 [00:12<00:00,  6.02s/it]

You can call me on  +33(0)6 12 34 56 78
{'phone': 0.8489881753921509} 



In [9]:
text_for_test=""" KIRED Nour Elhouda born 28-06-2000 in Saida, Algeria
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 54s {'birth': 0.9938105940818787, 'person': 0.9674139738082885, 'health': 0.5278267773489157}

100%|██████████| 1/1 [00:04<00:00,  4.64s/it]

 KIRED Nour Elhouda born 28-06-2000 in Saida, Algeria
birth checked
{'person': 0.8129440049330393, 'birth': 0.5158457029610872} 



In [52]:
text_for_test=""" KIRED Nour Elhouda lived in 4 allée du lieutenant Lucien Lafay, 31400 Toulouse
"""
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")
# translate
# 54s {'birth': 0.9938105940818787, 'person': 0.9674139738082885, 'health': 0.5278267773489157}

100%|██████████| 1/1 [00:04<00:00,  4.43s/it]

 KIRED Nour Elhouda lived in 4 allée du lieutenant Lucien Lafay, 31400 Toulouse
{'person': 0.7240406970183054, 'location': 0.5996545056502024} 



In [10]:
text_for_test="""je travaille depuis 9 ans en tant qu'assistante commerciale puis chargée de clientèle au sein de l'entreprise de mobilier et décoration pour enfants Nanelle, collaborant avec des clients français mais aussi internationaux. \
 Aujourd'hui, je suis à la recherche d'un nouveau déni ou la dimension internationale est très presente, notamment avec les pays hispaniques. Forte de mes expériences passées, je sais être à l'écoute des clients, gérer les situations de crise et j'aime travailler en équipe.\
 Fort en 3 ans d'expérience en tant que responsable des projets événementiels et marketing opérationnel et 2 ans en tant qu'office manager au sein d'une start-up européenne, j'ai appris à mener des missions très diverses qui m'fait \
 permis de développer une importance capacité d'adaptation à toutes les situations. Aujourd'hui je suis à la recherche d'une nouvelle expérience d'office manager dans une strat-up afin de mettre à profit mes competences pour offrir un environnement \
 de travail simulant à mes futurs collegues.\
 Mon nom est Laval JACQUIN, j'ai 38 ans, j'habite 29 bis chemin de Rivals,\
 mon numéro de téléphone est le +33652105579 et la plaque d'immatriculation de ma voiture est BP-152-FX, mon adresse mail est le l.jaquin@okp4.com. \
Issue a Civil Monetary Penalty notice \
Leading to a fine ranging up to £500,000\
Issue an Enforcement Notice \
A formal notice requiring an organisation or individual to take the action specified in the notice in order to bring about compliance \
with the Act and related laws. Failure to comply with a notice is a criminal offence.\
Request and agree an Undertaking with the organisation \
A formal undertaking can be given by an organisation to the ICO, committing the organisation to a particular course of action or \
otherwise achieving compliance. \
Mohandas Karamchand Gandhi est né le 2 octobre 1869 à Porbandar, dans l'actuel État du Gujarat, en Inde. Gandhi est né et a vécu \
toute sa vie en tant qu'hindou, mais dans une famille ouverte aux autres communautés religieuses, qu'elles soient jaïne, musulmane, ou parsie."""


pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for txt in  tqdm(nltk.sent_tokenize(text_for_test),total=len(nltk.sent_tokenize(text_for_test))):
        # text_translated=translate(txt)
        result_details=pipeline(txt, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        result=dict(sorted(result.items(), key=operator.itemgetter(1),reverse=True))
        pii_detected={name : result[name] for name in tresh(result)}
        print(txt)
        pii_detected=check(pii_detected,txt)
        if license_plate(txt):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")

# translate
# 2'50s No PII detected /No PII detected/No PII detected/No PII detected/No PII detected/{'person': 0.9767806768417359, 'phone': 0.935712973276774, 'location': 0.519710399210453, 'license_plate': 0.9}/{'person': 0.7851595520973206} /{'person': 0.6726939827203751}


 10%|█         | 1/10 [00:09<01:23,  9.23s/it]

je travaille depuis 9 ans en tant qu'assistante commerciale puis chargée de clientèle au sein de l'entreprise de mobilier et décoration pour enfants Nanelle, collaborant avec des clients français mais aussi internationaux.
No PII detected


 20%|██        | 2/10 [00:16<01:04,  8.04s/it]

Aujourd'hui, je suis à la recherche d'un nouveau déni ou la dimension internationale est très presente, notamment avec les pays hispaniques.
No PII detected


 30%|███       | 3/10 [00:23<00:51,  7.41s/it]

...

Failure to comply with a notice is a criminal offence.Request and agree an Undertaking with the organisation A formal undertaking can be given by an organisation to the ICO, committing the organisation to a particular course of action or otherwise achieving compliance.
No PII detected


 90%|█████████ | 9/10 [01:08<00:06,  6.77s/it]

Mohandas Karamchand Gandhi est né le 2 octobre 1869 à Porbandar, dans l'actuel État du Gujarat, en Inde.
birth checked
{'person': 0.7373657325903574, 'birth': 0.5222652237862349} 



100%|██████████| 10/10 [01:15<00:00,  7.52s/it]

Gandhi est né et a vécu toute sa vie en tant qu'hindou, mais dans une famille ouverte aux autres communautés religieuses, qu'elles soient jaïne, musulmane, ou parsie.
No PII detected


In [11]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df_fr['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        # text_translated=translate(sent)
        result_details=pipeline(sent, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        pii_detected={name : result[name] for name in tresh(result)}
        print(sent)
        pii_detected=check(pii_detected,sent)
        if license_plate(sent):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")

# translate
# 3'21s No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/{'person': 0.5827060759067535} /{'person': 0.880791163444519}

 10%|█         | 1/10 [00:06<01:00,  6.69s/it]

Je n'en parlerai pas, même s'il y a beaucoup à couvrir.
birth checked
{'birth': 0.510120808146894} 



 20%|██        | 2/10 [00:13<00:55,  6.98s/it]

Je ne vais pas parler de l'histoire de la ville parce qu'il y aurait trop de choses à dire.
birth checked
{'birth': 0.5003659430076368} 



 30%|███       | 3/10 [00:20<00:48,  6.89s/it]

...

Je ne sais pas pourquoi c'est arrivé.
No PII detected


 90%|█████████ | 9/10 [00:46<00:04,  4.46s/it]

Fannie Flono a dû reporter et n'est pas en mesure de nous raconter d'histoires aujourd'hui.
No PII detected


100%|██████████| 10/10 [00:51<00:00,  5.19s/it]

Fannie Flono est ici et elle va nous parler de ses histoires d'enfance de l'endroit ou elle a grandit, à Augusta, GA.
{'person': 0.6568525210022926} 



In [15]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        # text_translated=translate(sent)
        result_details=pipeline(sent, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        pii_detected={name : result[name] for name in tresh(result)}
        print(sent)
        pii_detected=check(pii_detected,sent)
        if license_plate(sent):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")

# translate
# 3'21s No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/{'person': 0.5827060759067535} /{'person': 0.880791163444519}

 10%|█         | 1/10 [00:07<01:09,  7.67s/it]

لن أتحدث عن ذلك ، على الرغم من أن هناك الكثير لتغطيته.
No PII detected


 20%|██        | 2/10 [00:15<01:02,  7.75s/it]

لن أتحدث عن تاريخ المدينة لأن هناك الكثير مما يمكن قوله حولها.
No PII detected


 30%|███       | 3/10 [00:20<00:46,  6.64s/it]

...

لا أعرف لماذا حدث هذا.
No PII detected


 90%|█████████ | 9/10 [00:56<00:06,  6.23s/it]

كان على فاني فلونو إعادة تعيين جدول الأعمال وهي غير قادرة على إخبارنا بأية قصص اليوم.
No PII detected


100%|██████████| 10/10 [01:04<00:00,  6.43s/it]

فاني فلونو هنا وستتحدث إلينا عن قصص طفولتها التي نشأت في أوغستا ، جورجيا.
No PII detected


In [11]:

pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passeport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number")]
for row in  tqdm((df_en['sentence2'][10:20]),total=10):
    txt=nltk.sent_tokenize(row)
    for sent in txt:
        # text_translated=translate(sent)
        result_details=pipeline(sent, candidate_labels, multi_label=True)
        result_details=dict(zip(result_details['labels'],result_details['scores']))
        print(result_details)
        result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
        pii_detected={name : result[name] for name in tresh(result)}
        print(sent)
        pii_detected=check(pii_detected,sent)
        if license_plate(sent):
                pii_detected["license_plate"]=0.9
        if pii_detected!={}:
                print(pii_detected,'\n')
        else:
                print("No PII detected")

# translate
# 3'21s No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/No PII detected/{'person': 0.5827060759067535} /{'person': 0.880791163444519}

 10%|█         | 1/10 [00:04<00:41,  4.65s/it]

{'private': 0.9945345520973206, 'non privé': 0.8859943747520447, 'informations non privées': 0.8283578157424927, 'non personnel': 0.7515348196029663, 'informations non personnelles': 0.6453291177749634, 'informations privées': 0.5549554228782654, 'informations personnelles': 0.42160549759864807, 'données non privées': 0.4158676862716675, 'données non personnelles': 0.3695247173309326, 'personnel': 0.289412260055542, 'dossiers médicaux': 0.28686320781707764, ' médicales': 0.26644623279571533, 'numéro de dossier fiscal': 0.2628457546234131, 'noms des actes médicaux': 0.2276528924703598, 'adresse postale': 0.21384915709495544, 'données privées': 0.15129037201404572, 'private postal address': 0.14087291061878204, 'numéro de carte de crédit': 0.1396367847919464, 'numéros de téléphone': 0.12542717158794403, 'location adress': 0.12241769582033157, 'name and family name': 0.11758584529161453, 'Medical Procedure identification codes': 0.11241283267736435, "codes d'identification des actes médic

 20%|██        | 2/10 [00:11<00:45,  5.74s/it]

{'private': 0.9496273398399353, 'non privé': 0.5999212265014648, 'informations non privées': 0.5541610717773438, 'informations non personnelles': 0.3458573520183563, 'données non privées': 0.32920828461647034, 'données non personnelles': 0.29114991426467896, 'adresse postale': 0.16717331111431122, 'numéro de carte de crédit': 0.10822229087352753, 'non personnel': 0.10756813734769821, 'données privées': 0.08126766979694366, 'informations privées': 0.07091555744409561, 'données personnelles': 0.06890197098255157, 'courrier electronique': 0.053239647299051285, 'Nom et prenom': 0.05191493034362793, 'numéro de dossier fiscal': 0.05059781298041344, 'noms des actes médicaux': 0.04391191527247429, ' médicales': 0.04333028197288513, 'location adress': 0.03904550150036812, 'private postal address': 0.03769258037209511, 'données de santé': 0.03641970828175545, 'numero de permis': 0.03249133750796318, 'dossiers médicaux': 0.0230263639241457, 'informations personnelles': 0.017905287444591522, 'numé

 30%|███       | 3/10 [00:17<00:43,  6.23s/it]

...

{'private': 0.6107561588287354, 'données non personnelles': 0.48805466294288635, 'dossiers médicaux': 0.4810478985309601, 'informations non privées': 0.4487193524837494, 'adresse postale': 0.4459368884563446, 'courrier electronique': 0.43433690071105957, 'données non privées': 0.4328211843967438, 'informations non personnelles': 0.3983699083328247, 'données privées': 0.3916398584842682, 'numéro de carte de crédit': 0.3799601197242737, ' médicales': 0.36390790343284607, 'données personnelles': 0.36032089591026306, 'informations privées': 0.3472030758857727, 'noms des actes médicaux': 0.34445348381996155, 'non privé': 0.3411942720413208, 'numéros de téléphone': 0.33702683448791504, 'Nom et prenom': 0.3361336886882782, 'non personnel': 0.32249143719673157, 'données de santé': 0.32034727931022644, 'numéro de dossier fiscal': 0.3186151683330536, 'personnel': 0.30490347743034363, 'informations personnelles': 0.24850711226463318, 'email': 0.22730693221092224, 'numero de permis': 0.19940027594

 90%|█████████ | 9/10 [00:53<00:05,  5.98s/it]

{'personnel': 0.9679994583129883, "person's name": 0.9094294309616089, 'private': 0.8992266058921814, 'name and family name': 0.8893520832061768, 'informations non privées': 0.8819746971130371, 'informations non personnelles': 0.7601985931396484, 'non personnel': 0.7124296426773071, 'données non privées': 0.6918005347251892, 'informations personnelles': 0.6738260984420776, 'non privé': 0.5765244364738464, 'informations privées': 0.5340037941932678, 'données non personnelles': 0.5275666117668152, 'adresse postale': 0.48060885071754456, 'numéro de carte de crédit': 0.3723325729370117, 'données privées': 0.32900795340538025, 'location adress': 0.311132550239563, 'numéro de dossier fiscal': 0.2930653989315033, ' médicales': 0.2516530156135559, 'dossiers médicaux': 0.24321377277374268, 'numero de permis': 0.22124312818050385, 'données personnelles': 0.20110514760017395, 'numéros de téléphone': 0.1886569857597351, 'noms des actes médicaux': 0.1699378490447998, 'phone number': 0.1605224311351

100%|██████████| 10/10 [00:59<00:00,  5.93s/it]

{'name and family name': 0.9946346879005432, "person's name": 0.9725118279457092, 'location adress': 0.8092573881149292, 'informations personnelles': 0.40207475423812866, 'non personnel': 0.38204821944236755, 'informations privées': 0.2999437749385834, 'données privées': 0.2721082270145416, 'adresse postale': 0.24156689643859863, 'informations non privées': 0.18175002932548523, 'données non personnelles': 0.1430959701538086, 'données non privées': 0.13041952252388, 'données de santé': 0.12517650425434113, 'Nom et prenom': 0.12435545027256012, 'birthday': 0.11698594689369202, 'données personnelles': 0.11499765515327454, 'informations non personnelles': 0.11405400931835175, 'non privé': 0.08861041069030762, 'numéro de dossier fiscal': 0.0630500391125679, 'private postal address': 0.0530509315431118, 'numero de permis': 0.05233291536569595, 'phone number': 0.04713309556245804, 'courrier electronique': 0.04251214489340782, 'numéro de carte de crédit': 0.041952479630708694, 'noms des actes 